# The Hidden Markov Model
author: Tom Stone <tomstone@stanford.edu>\
License: BSD (3-clause)

We will use Numpy for our numerical computation

In [ ]:
import numpy as np

Objects used consistently throughout the functions:

The state at time $t$ is given by $q_t$, and the set of all states is $\mathbf{Q}$. The state transition matrix is denoted $A = [a_{ij}]_{i,j=1}^{N,N}$ where $a_{ij} = \mathbb{P}[q_{t+1}=j|q_t=i]$. The observation at time $t$ is given by $O_t$, and the set of all observations is $\mathbf{O}$. The emission probabilities are $b_j(O_t) = \mathbb{P}[O_t|q_t=j]$. The prior probability of the initial states are $\pi = \{\pi_i\}$
<!-- 
POTENTIALLY CONFUSING SIDENOTE:
It turns out that for the somata implementation it is more useful to think of the $b$ matrix as being $N\times T$ instead of $N\times M$ (where $M$ is the number of states), because the only emission probability you use at time $t$ is $b_j(O_t)$, you don't use any other $b_j(m)$ for that time. -->
# Problem 1: Observable Markov Model

State 1: precipitation\
State 2: Cloudy\
State 3: Sunny

Transition matrix:
\begin{align*}
a_{11}\hspace{1mm} a_{12}\hspace{1mm} a_{13}\\
a_{21}\hspace{1mm} a_{22}\hspace{1mm} a_{23}\\
a_{31}\hspace{1mm} a_{32}\hspace{1mm} a_{33}
\end{align*}

Q1: given that today is sunny, what is the probability the next 7 days will be sunny, but the 8th won't?

Q2: Given that today is sunny, what is the *expected* number of consecutive days it will be sunny?








# Problem 2: The forward-backward algorithm(s)

Q3: A pen and paper implementation of the forward & backward algorithms

## The discrete-valued HMM forward algorithm

Want to calculate
$$\mathbb{P}[\mathbf{O}=O_1O_2...O_T|\lambda]$$

We use the inductive value $\alpha_t(i)$
$$\alpha_t(i) = \mathbb{P}[O_1O_2...O_t,q_t = S_i|\lambda]$$

Which is initialized as 
$$\forall\text{ (forall) } j\hspace{1mm}\alpha_1(j) = \pi_jb_j(O_1)$$
where $\pi_j$ is the prior on state $j$, $b_j(O_1)$ is the probability of observing $O_1$ given state $j$, and $A=[a_{ij}]$ is the state transition matrix.

The inductive step:

$$\alpha_{t+1}(j) = \left[\sum_{i=1}^N\alpha_t(i)a_{ij}\right]b_j(O_{t+1})$$

Termination:

$$\mathbb{P}[\mathbf{O}|\lambda]=\sum_{i=1}^N\alpha_T(i)$$

### Function `forward(A, b, pi, obs)`

Your task is to write a function that takes in the state transition matrix `A`, emission probabilities `b`, state priors `pi`, and observations `obs`, and returns the likelihood $\mathbb{P}[\mathbf{O}|\lambda]$ calculated with the forward algorithm, as well as the $\alpha_t$'s

In [ ]:
def forward(A, b, pi, obs):
    """
    inputs: 
        A: NxN transition matrix, rows sum to 1
        b: NxM emission probabilities, rows sum to 1
        pi: N priors on hidden states
        obs: T observations
    returns:
        likelihood: likelihood P[O|lambda]
        alpha: recursive object calculated in forward algorithm
    """
    #############
    # sanity checks for data shapes
    #
    N = A.shape[0] # number of states
    assert N == A.shape[1], "A is not square"
    assert N == b.shape[0], "b dimension does not match A"
    assert N == pi.shape[0], "pi dimension does not match A"

    M = b.shape[1]
    T = len(obs)
    #
    #############

    # step 1: 
    # preallocate alpha matrix (what is its shape?)
    alpha = 


    # step 1.1: 
    # initialize alpha
    alpha[:,0] = 


    # step 2: induction 
    for t in range(1,T):
        # fill in here
        # can do induction with another loop
        # or with a careful matrix multiplication
        


    # step 3: termination
    likelihood = 




    return likelihood, alpha


## Discrete-valued HMM backward algorithm

Again, we want to calculate
$$\mathbb{P}[\mathbf{O}|\lambda]$$

The inductive value this time is:
$$\beta_t(i) = \mathbb{P}[O_{t+1}...O_T|q_t=S_i,\lambda]$$

The base case is simpler than in the $\alpha$ case:
$$\forall i\hspace{1mm}\beta_T(i) = 1$$

But the recursive formula is just a tad more complicated (multiplication by $b$ is now on the inside)
$$\beta_t(i) = \sum_{j=1}^Na_{ij}\beta_{t+1}(j)b_j(O_{t+1})$$

The termination is:
$$\mathbb{P}[\mathbf{O}|\lambda] = \sum_{j=1}^N\pi_j\beta_1(j)b_j(O_1)$$

### Function `backward(A, b, pi, obs)`

Your task is to write a function that takes in the state transition matrix `A`, emission probabilities `b`, state priors `pi`, and observations `obs`, and returns the likelihood $\mathbb{P}[\mathbf{O}|\lambda]$ computed with the backwards algorithm, as well as the $\beta_t$'s

In [ ]:
def backward(A, b, pi, obs):
    """
    inputs: 
        A: NxN transition matrix, rows sum to 1
        b: NxM emission probabilities, rows sum to 1
        pi: N priors on states
        obs: T observations
    returns:
        likelihood: likelihood P[O|lambda]
        beta: recursive object calculated in backward algorithm
    """
    #############
    # sanity checks for data shapes
    #
    N = A.shape[0] # number of states
    assert N == A.shape[1], "A is not square"
    assert N == b.shape[0], "b dimension does not match A"
    assert N == pi.shape[0], "pi dimension does not match A"

    M = b.shape[1]
    T = len(obs)
    #
    #############

    # step 1: preallocate beta
    beta = 

    # step 1.1: initialize beta_T
    beta[:,-1] = 
        

    # step 2: inductive step
    for t in reversed(range(0,T-1)):
        # fill in here
        # can do induction with another loop
        # or with a careful matrix multiplication
    


    # step 3: termination
    likelihood = 



    return likelihood, beta




In [ ]:
# check your paper calculations
forward(
    np.array([[0.9, 0.1], [0.2, 0.8]]), # A
    np.array([[0.7, 0.3], [0.4, 0.6]]), # b
    np.array([0.5, 0.5]),               # pi
    np.array([0,0,1,0])                 # obs
)

In [ ]:
backward(
    np.array([[0.9, 0.1], [0.2, 0.8]]), # A
    np.array([[0.7, 0.3], [0.4, 0.6]]), # b
    np.array([0.5, 0.5]),               # pi
    np.array([0,0,1,0])                 # obs
)

In [ ]:
def forward(A, b, pi, obs):

    N = A.shape[0] # number of states
    assert N == A.shape[1], "A is not square"
    assert N == b.shape[0], "b dimension does not match A"
    assert N == pi.shape[0], "pi dimension does not match A"

    M = b.shape[1]
    T = len(obs)

    alpha = np.zeros((N, T))
    alpha[:,0] = pi * b[:,obs[0]]

    for t in range(1,T):
        for j in range(M):
            # solution #1
            alpha[j,t] = sum([al * a for (al,a) in zip(alpha[:,t-1], A[:,j])]) * b[j,obs[t]]

            # solution #2
            alpha[j,t] = np.dot(alpha[:,t-1], A[:,j]) * b[j,obs[t]]

        # solution #3
        alpha[:,t] = (np.transpose(A) @ alpha[:,t-1]) * b[:, obs[t]]
    
    return sum(alpha[:,-1]), alpha


def backward(A, b, pi, obs):
    N = A.shape[0] # number of states
    assert N == A.shape[1], "A is not square"
    assert N == b.shape[0], "b dimension does not match A"
    assert N == pi.shape[0], "pi dimension does not match A"

    M = b.shape[1]
    T = len(obs)

    beta = np.ones((N,T))

    for t in np.arange(T-2, -1, step=-1): # reversed(range(0,T-1))
        for i in range(M):
            # solution #1
            beta[i,t] = sum([a * bet * b for (a, bet, b) in zip(A[i,:], beta[:,t+1], b[:, obs[t+1]])])

            # solution #2
            beta[i,t] = np.dot(A[i,:], beta[:,t+1] * b[:,obs[t+1]])

        # solution #3
        beta[:,t] = A @ (beta[:,t+1] * b[:,obs[t+1]])

    return np.dot(pi * beta[:,0], b[:, obs[0]]), beta


# Problem 3: Optimal state estimation
## Locally optimal state estimation
As we discussed in class, there are several ways to define the 'optimal' state sequence. We will start with the classes that are individually most likely.

The recursive object calculated here is;

$$\gamma_t(i) = \mathbb{P}[q_t=S_i|\mathbf{O},\lambda]$$

Rewriting using Bayes' rule, 

$$\gamma_t(i) = \frac{\mathbb{P}[q_t=S_i,\mathbf{O}|\lambda]}{\mathbb{P}[\mathbf{O}|\lambda]} = \frac{\alpha_t(i)\beta_t(i)}{\sum_{i=1}^N \alpha_t(i)\beta_t(i)}$$

The most likely state at time $t$ is then:
$$q_t^* = \argmax_{1\leq i\leq N}[\gamma_i(t)]$$

### function `locally_optimal_state_sequence(A, b, pi, obs)`

Your task is to write a function that takes in the state transition matrix `A`, emission probabilities `b`, state priors `pi`, and observations `obs`, and returns the sequence of states that are individually optimal

In [ ]:
def locally_optimal_state_sequence(A, b, pi, obs):
    """
    inputs: 
        A: NxN transition matrix, rows sum to 1
        b: NxM emission probabilities, rows sum to 1
        pi: N priors on states
        obs: T observations
    returns:
        maxs: sequence of locally optimal states
        gamma: recursive object calculated in locally optimal algorithm
    """
    #############
    # sanity checks for data shapes
    #
    N = A.shape[0] # number of states
    assert N == A.shape[1], "A is not square"
    assert N == b.shape[0], "b dimension does not match A"
    assert N == pi.shape[0], "pi dimension does not match A"

    M = b.shape[1]
    T = len(obs)
    #
    #############
    
    # get alpha, beta
    _, alpha = forward(A, b, pi, obs)
    _, beta = backward(A, b, pi, obs)

    # compute gamma
    gamma = 



    # compute argmax
    maxs = 



    return maxs, gamma

## Globally optimal state estimation

If you want to optimize $\mathbb{P}[\mathbf{Q}|\mathbf{O},\lambda]$ over $\mathbf{Q}$, by Bayes rule it is equivalent to optimizing $\mathbb{P}[\mathbf{Q, O}|\lambda]$

The recursive objects here are:

$$\delta_t(i) = \max_{q_1...q_{t-1}}\mathbb{P}[q_1...q_{t-1},q_t=i,O_1...O_t|\lambda]\cdot b_j(O_{t+1})$$
$$\psi_t(i) = \argmax_{q_1...q_{t-1}}\mathbb{P}[q_1...q_{t-1},q_t=i,O_1...O_t|\lambda]$$

The base case is:

$$\delta_1(i) = \pi_ib_i(O_1)$$
$$\psi_1(i) = 0\text{ (not actually used)}$$

The recursive formula is:

$$\delta_t(j) = \max_i[\delta_{t-1}(i)a_{ij}]\cdot b_j(O_t)$$
$$\psi_{t+1}(j) = \argmax_{i}[\delta_{t-1}(i)a_{ij}]$$

The Termination is 
$$P^* = \max_i[\delta_T(i)]$$
$$q_T^* = \argmax_i[\delta_T(i)]$$

And then you backtrack along the path with the formula

$$q_t^* = \psi_{t+1}(q_{t+1}^*)$$

### function `globally_optimal_state_sequence(A, b, pi, obs)`

Your task is to write a function that takes in the state transition matrix `A`, emission probabilities `b`, state priors `pi`, and observations `obs`, and returns the probability of the sequence $P^*$, the sequence itself $q^*$, and $\delta$, the recursive object.

In [ ]:
def globally_optimal_state_sequence(A, b, pi, obs):
    """
    inputs: 
        A: NxN transition matrix, rows sum to 1
        b: NxM emission probabilities, rows sum to 1
        pi: N priors on states
        obs: T observations
    returns:
        Pstar: probability of qstar sequence
        qstar: optimal sequence of states
        delta: recursive object calculated in globally optimal algorithm
    """
    #############
    # sanity checks for data shapes
    #
    N = A.shape[0] # number of states
    assert N == A.shape[1], "A is not square"
    assert N == b.shape[0], "b dimension does not match A"
    assert N == pi.shape[0], "pi dimension does not match A"

    M = b.shape[1]
    T = len(obs)
    #
    #############

    # step 1: preallocation
    delta = 
    psi = 


    # step 1.1: initialization 
    delta[:,0] = 
    psi[:,0] = 



    # step 2: recursion
    for t in range(1,T):
        # fill in here
        # can do induction with another loop
        # or with a VERY VERY careful matrix multiplication




    # step 3: termination, init. qstar
    Pstar = 
    qstar = np.zeros(, dtype=np.int64) # this data type will help you avoid errors, trust me
    qstar[-1] = 

    

    # step 4: path backtracking
    for t in reversed(range(0,T-1)):
        # chill loop


    return Pstar, qstar, delta


In [ ]:
globally_optimal_state_sequence(
    np.array([[0.9, 0.1], [0.2, 0.8]]), # A
    np.array([[0.7, 0.3], [0.4, 0.6]]), # b
    np.array([0.5, 0.5]), # pi
    np.array([1,1,0,0]) # obs
)

In [ ]:
def locally_optimal_state_sequence(A, b, pi, obs):
    _, alpha = forward(A, b, pi, obs)
    _, beta = backward(A, b, pi, obs)

    gamma = alpha * beta
    # normalize columns (don't actually need to unless you explicitly want gamma)
    tgamma = gamma.sum(0, keepdims=True)
    gamma /= tgamma
    maxs = np.argmax(gamma, axis=0, keepdims=True)

    return maxs, gamma


def globally_optimal_state_sequence(A, b, pi, obs):
    N = A.shape[0] # number of states
    assert N == A.shape[1], "A is not square"
    assert N == b.shape[0], "b dimension does not match A"
    assert N == pi.shape[0], "pi dimension does not match A"

    M = b.shape[0]
    T = len(obs)

    delta = np.zeros((N, T))
    delta[:,0] = pi * b[:, obs[0]]
    
    psi = np.zeros((N, T))

    # forward
    for t in range(1,T):
        for j in range(M):
            # option 1:
            temp = [d * a for (d,a) in zip(delta[:,t-1], A[:,j])]
            # option 2:
            temp = delta[:, t-1] * A[:,j]

            delta[j,t] = np.max(temp) * b[j,obs[t]]
            psi[j,t] = np.argmax(temp)

        # # option 3:
        temp = delta[:,t-1][:,None] * A # avoid numpy auto coercing to row vector??, thus messing up mult.
        # print(temp)
        delta[:,t] = np.max(temp, axis=0) * b[:,obs[t]]
        psi[:,t] = np.argmax(temp, axis=0)
    
    Pstar = np.max(delta[:,-1])
    qstar = np.zeros((T), dtype=np.int64)
    qstar[-1] = np.argmax(delta[:,-1])

    # path backtracking
    for t in reversed(range(0, T-1)):
        qstar[t] = psi[qstar[t+1], t+1]

    return Pstar, qstar, delta

In [ ]:
# example of the coercion
print(np.array([1,2])[:,None] * np.array([[1,2],[3,4]]))
print(np.array([1,2]) * np.array([[1,2],[3,4]]))